In [357]:
import math
import numpy as np
import plotly.graph_objects as go
import scipy.integrate as sci

class Constants:
    # Количество фракций
    FACTIONS = 1000
    # Всего фракций
    ALL_FACTIONS = 3e6

    # Число Авогадро
    NA = 6.022e23

    # Молекулярные массы, кг/моль
    # На вики ММ стирола равен 0,10415 кг/моль
    MMs = 0.092  # стирол
    MMd = 0.054  # дивинил

    # Постоянные константы скорости инициирования и роста цепи [л/(мин*моль)]
    Ki0, Ks0, Kd0 = 0.835e10, 5.76e11, 2e12

    # Энергия активации инициирования и роста полистирольных цепей [Дж/моль]
    Ei, Es, Ed = 59962, 71184, 80850

    # Универсальная газовая постоянная
    R = 8.32

    # Коэффициенты теплопередачи [Дж/(м2*К*мин)]
    A, B, C = 14.17, 1.007, 10.3823

    # Параметр зависимости, учитывающей влияние концентрации полимера на порядок реакции по катализатору
    ws = -0.4142
    #
    wd = -0.184
    #
    b = 3.77

    # Площадь поверхности теплообмена в реакторе, m^2
    Fst = 42.0
    # Коэффициент тепловедение стирола и дивинила
    Kted, Ktes = 80.9, 74.87

    # Масса аппарата (реактора), кг
    Map = 16820
    # Теплоёмкости (Дж/(кг*К))
    Cap = 0.46  # материал аппарата
    Cpm = 1.8436  # реакционная масса
    # Плотность реакционной массы, кг/л
    dpm = 0.763
    # Теплоёмкость и плотность хладагента
    Ch, ph = 4.19, 1

class BaseExperiment(Constants):
    """Базовый класс для вычисления производных значений экспериментов."""

    def __init__(self, Ghd1, Ghd2, Ghs, Jk, Md1, Md2, Mr, Ms, Pd, Pr, Ps, T0, T1, Texp1, Texp2, Th, Vk, ms0, ng):
        self.Ghd1 = Ghd1
        self.Ghd2 = Ghd2
        self.Ghs = Ghs
        self.Jk = Jk
        self.Md1 = Md1
        self.Md2 = Md2
        self.Mr = Mr
        self.Ms = Ms
        self.Pd = Pd
        self.Pr = Pr
        self.Ps = Ps
        self.T0 = T0
        self.T1 = T1
        self.Texp1 = Texp1
        self.Texp2 = Texp2
        self.Th = Th
        self.Vk = Vk
        self.ms0 = ms0
        self.ng = ng

    @property
    def Vd1(self):
        """Объём дивинила на стадии 1."""
        return self.Md1 / self.Pd

    @property
    def Vd2(self):
        """Объём дивинила на стадии 2."""
        return self.Md2 / self.Pd

    @property
    def Vr(self):
        """Объём растворителя."""
        return self.Mr / self.Pr

    @property
    def Vs(self):
        """Объём стирола."""
        return self.Ms / self.Ps

    @property
    def Vt(self):
        """Объём реакционной смеси."""
        return self.Vk + self.Vs + self.Vr

    @property
    def Jacd1(self):
        """Концентрация активных центров на стадии 1."""
        return self.Jk * (self.Vt / (self.Vt + self.Vd1))

    @property
    def Jacd2(self):
        """Концентрация активных центров на стадии 1."""
        return self.Jk * (self.Vt / (self.Vt + self.Vd2))

    @property
    def L(self):
        """Степень заполнения."""
        return (self.Vt - 3000) / 28500

    # Можно использовать и эту формулу, будет то же самое, но точнее. Взято из диссертации стр. 66
    # @property
    # def ms0(self):
    #     """Объём дивинила на стадии 1."""
    #     return self.Ps*self.Vs / (self.MMs * self.Vt)

    @property
    def md10(self):
        """Начальная концентрация дивинила на первой стадии."""
        return (self.Pd * self.Vd1) / (self.MMd * (self.Vt + self.Vd1))

    @property
    def md20(self):
        """Начальная концентрация дивинила на второй стадии."""
        return (self.Pd * self.Vd2) / (self.MMd * (self.Vt + self.Vd2))

class Experiment1(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1666, Ghd2=1866, Ghs=1500,
            Jk=0.00478,
            Md1=1765, Md2=837, Mr=11400, Ms=1140,
            Pd=0.638, Pr=0.748, Ps=0.91,
            T0=310.4, T1=307.4,
            Th=290, Vk=139.2, ms0=0.745, ng=0.125,
            Texp1=[310.4, 310.9, 311.5, 312.4, 313.6, 315.2, 316.7, 318.5, 320.7, 322.3, 324.4, 326.6, 328.2, 329.8, 331.0, 332.0, 333.3, 333.3, 333.7, 333.7, 333.7, 333.5, 333.4, 333.2, 332.8, 332.6, 332.3, 332.0, 331.7, 331.4],
            Texp2=[307.4, 307.5, 307.6, 307.7, 308.2, 308.8, 309.6, 310.5, 311.5, 312.5, 313.7, 315.0, 316.5, 318.2, 320.2, 322.6, 325.6, 329.2, 334.6, 341.6, 353.0, 363.2, 368.8, 370.8, 371.4, 370.8],
        )
class Experiment2(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1666, Ghd2=1366, Ghs=2000,
            Jk=0.0045,
            Md1=1687, Md2=715, Mr=11140, Ms=1080,
            Pd=0.637, Pr=0.748, Ps=0.916,
            T0=309.6, T1=310.4,
            Th=289.3, Vk=139.2, ms0=0.7304, ng=0.129,
            Texp1=[309.6, 310.0, 310.3, 310.7, 311.4, 312.0, 313.0, 314.0, 315.0, 316.2, 317.5, 319.0, 320.4, 322.3, 323.7, 325.4, 326.8, 327.9, 329.0, 330.0, 330.7, 331.2, 331.6, 331.8, 332.0, 332.0, 331.9, 331.9, 331.6, 331.4],
            Texp2=[310.4, 310.6, 311.3, 312.2, 313.3, 314.4, 315.9, 317.4, 319.4, 321.4, 323.9, 327.3, 332.1, 338.6, 349.6, 361.4, 368.9, 372.7, 373.9, 373.9, 373.5, 372.6, 371.8],
        )

class Experiment3(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1366, Ghd2=1366, Ghs=2000,
            Jk=0.00416,
            Md1=1701, Md2=715, Mr=11370, Ms=1085,
            Pd=0.637, Pr=0.739, Ps=0.92,
            T0=312.7, T1=309.5,
            Th=288.5, Vk=139.2, ms0=0.7129, ng=0.13,
            Texp1=[312.7, 312.9, 313.4, 314.0, 315.0, 316.0, 317.3, 319.2, 320.8, 322.4, 324.8, 326.7, 328.6, 330.1, 331.4, 332.7, 333.6, 334.3, 334.6, 334.3, 334.2, 333.9, 333.5, 333.3, 333.0, 332.7, 332.4, 332.2, 331.8, 331.5],
            Texp2=[309.5, 309.6, 309.6, 309.6, 311.6, 312.6, 313.6, 314.9, 316.3, 317.8, 319.5, 321.4, 323.7, 326.8, 330.4, 336.1, 344.1, 354.9, 365.3, 370.0, 372.6, 373.6, 373.9],
        )

class Experiment4(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1666, Ghd2=1666, Ghs=1666,
            Jk=0.00361,
            Md1=1664, Md2=722, Mr=11500, Ms=1081,
            Pd=0.635, Pr=0.75, Ps=0.919,
            T0=309.3, T1=313.3,
            Th=290, Vk=139.2, ms0=0.7106, ng=0.147,
            Texp1=[309.3, 309.4, 309.6, 310.0, 310.3, 310.9, 311.6, 312.2, 313.3, 314.2, 315.7, 317.0, 318.3, 319.8, 321.6, 323.1, 324.5, 326.3, 327.4, 328.2, 329.2, 329.9, 330.4, 330.8, 331.1, 331.1, 331.1, 331.0, 330.9, 330.7],
            Texp2=[313.3, 314.3, 315.4, 316.7, 318.5, 320.5, 323.2, 325.8, 329.5, 335.9, 343.8, 353.9, 361.3, 367.4, 371.0, 373.1, 373.5, 373.8, 373.6, 373.1, 372.6],
        )

class Experiment5(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1866, Ghd2=1866, Ghs=1500,
            Jk=0.0041,
            Md1=1558, Md2=663, Mr=11270, Ms=998,
            Pd=0.637, Pr=0.744, Ps=0.921,
            T0=313.0, T1=314.5,
            Th=296, Vk=139.2, ms0=0.6683, ng=0.129,
            Texp1=[313.0, 313.2, 313.6, 314.2, 315.6, 316.7, 318.2, 320.3, 322.9, 325.4, 328.2, 330.0, 331.0, 331.7, 333.9, 334.5, 334.9, 335.1, 335.2, 335.1, 334.9, 334.7, 334.5, 334.3, 334.1, 333.9, 333.7, 333.4, 333.1, 332.8],
            Texp2=[314.5, 315.7, 317.2, 318.7, 320.6, 323.2, 326.1, 329.7, 334.6, 342.1, 353.4, 364.3, 370.2, 372.6, 373.2, 372.9, 372.0, 371.1, 370.2, 369.4, 368.6],
        )

class Experiment6(BaseExperiment):
    def __init__(self):
        super().__init__(
            Ghd1=1166, Ghd2=1166, Ghs=1500,
            Jk=0.00402,
            Md1=1702, Md2=710, Mr=11290, Ms=1085,
            Pd=0.637, Pr=0.739, Ps=0.92,
            T0=311.2, T1=309.4,
            Th=288.5, Vk=139.2, ms0=0.7168, ng=0.137,
            Texp1=[311.2, 311.4, 311.8, 312.6, 313.4, 314.4, 315.5, 316.6, 317.7, 319.4, 321.0, 322.4, 324.2, 325.7, 327.4, 328.8, 330.1, 331.0, 332.0, 332.5, 333.1, 333.5, 333.7, 333.8, 333.9, 333.8, 333.7, 333.5, 333.5, 333.1],
            Texp2=[309.4, 309.6, 309.8, 310.2, 310.8, 312.1, 313.1, 314.3, 315.7, 317.1, 318.9, 321.2, 323.7, 326.9, 331.0, 337.9, 346.8, 356.4, 363.0, 368.1, 371.2, 372.7, 373.3],
        )

Константы времени моделирования

In [358]:
# Шаг дискретизации времени
TIMESTEP = 0.1

Блок 1:

Математическая модель

In [359]:

def kimpt(t, y, e: BaseExperiment):
    prevXi, prevXs, prevT = y[0], y[1], y[2]
    prevP = y[3:]

    Ki = e.Ki0 * math.pow(math.e, -e.Ei/(e.R*prevT))
    Ks = e.Ks0 * math.pow(math.e, -e.Es/(e.R*prevT))
    Cp = (e.Ms * prevXs) / e.Mr
    Kf = e.A - e.B * math.pow(math.e, e.C * Cp)

    dXi = Ki * e.ms0 * (1 - prevXi) * (1 - prevXs)
    dXs = (Ki * e.Jk * (1 - prevXi) + Ks * (math.pow(e.Jk * prevXi, 0.5 + e.ws*Cp))) * (1 - prevXs)

    numeratorLeft = (e.Ktes * dXs * e.ms0 * e.Vt)
    numeratorRigth = (Kf * e.Fst * e.L * (prevT - e.Th) * e.Ghs * e.Ch * e.ph)/(Kf * e.L * e.Fst + e.Ghs * e.Ch * e.ph)
    denominator = (e.Map * e.Cap) + (e.Vt * e.dpm * e.Cpm)
    dT = (numeratorLeft - numeratorRigth) / denominator

    dP = [0.0 for _ in range(e.FACTIONS)]
    ms = e.ms0 * (1 - prevXs)
    ks = Ks/math.pow(e.Jk, 0.5-e.ws*Cp)
    ki = Ki
    J = e.Jk * (1 - prevXi)

    for i in range(e.FACTIONS):
        if i == 0:
            newP = -ks * ms * prevP[0] + ki * ms * J
        elif i == e.FACTIONS-1:
            newP = ks * ms * prevP[i-1]
        else:
            newP = -ks * ms * prevP[i] + ks * ms * prevP[i-1]
        dP[i] = newP
    return [dXi, dXs, dT] + dP

def check_coef(t, y, terminal=False):
    prevXi, prevXs = y[0], y[1]
    return prevXs - prevXi

class Rolf:
    y = np.array(())
    t = np.array(())

def RKSolver(f, time, y0, args):
    t_values = [time[0]]
    y_values = [y0]
    t = time[0]
    y = np.array(y0)
    h = TIMESTEP
    while t < time[1]:
        if t + h > time[1]:
            h = time[1] - t

        k1 = h * np.array(f(t, y, *args))
        k2 = h * np.array(f(t + h/2, y + k1/2, *args))
        k3 = h * np.array(f(t + h/2, y + k2/2, *args))
        k4 = h * np.array(f(t + h, y + k3, *args))

        y = y + (k1 + 2*k2 + 2*k3 + k4) / 6
        t = t + h
        t_values.append(t)
        y_values.append(y.tolist())

    rol = Rolf()
    rol.y = np.array(y_values).T
    rol.t = np.arange(int(time[0]), int(time[1]), TIMESTEP)

    return rol

def block1(e: BaseExperiment):
    # Время интегрирования
    timeStart = 0.0
    timeEnd = 151

    check_coef.terminal = True
    check_coef.direction = 1
    y0 = [0, 0, e.T0] + [0 for _ in range(e.FACTIONS)]
    sol = sci.solve_ivp(
         kimpt,
         [timeStart, timeEnd],
         y0,
         method="DOP853",
         args=(e,),
         events=check_coef)

    # sol = RKSolver(kimpt, [timeStart, timeEnd], y0, args=(e,))

    print(sol.y[0][-1], sol.y[1][-1], sol.y[2][-1], [ar[-1] for ar in sol.y[3:]], sep="\t| ")

    dt = (sol.t[-1]-sol.t[0])/len(sol.t)
    print(f'Отклонение температуры на стадии 1: {sum([(sol.y[2][int(i/dt)]-e.Texp1[i]) for i in range(len(e.Texp1))])}')
    return sol

In [360]:
def kimpt2(t, y, e: BaseExperiment):
    prevXd, prevT = y[0], y[1]
    prevP = y[2:]

    cps = e.Ms / (e.Md1 + e.Ms + e.Mr)
    Cp = cps + (e.Md1 * prevXd) / e.Mr
    Kd = e.Kd0 * (1 - e.b * Cp) * math.pow(math.e, -e.Ed / (e.R * prevT))
    Kf = e.A - e.B * math.pow(math.e, e.C * Cp)

    dXd = (Kd * math.pow(e.Jacd1, 0.25 + e.wd*Cp)) * (1 - prevXd)

    numeratorLeft = (e.Kted * dXd * e.md10 * e.Vt)
    numeratorRigth = (Kf * e.Fst * e.L * (prevT - e.Th) * e.Ghd1 * e.Ch * e.ph)/(Kf * e.L * e.Fst + e.Ghd1 * e.Ch * e.ph)
    denominator = (e.Map * e.Cap) + (e.Vt * e.dpm * e.Cpm)
    dT = (numeratorLeft - numeratorRigth) / denominator

    dP = [0.0 for _ in range(e.FACTIONS)]
    md = e.md10 * (1 - prevXd)
    kd = Kd/math.pow(e.Jacd1, 0.75-e.wd*Cp)
    for i in range(e.FACTIONS):
        if i == 0:
            newP = -kd * md * prevP[0]
        elif i == e.FACTIONS-1:
            newP = kd * md * prevP[i-1]
        else:
            newP = -kd * md * prevP[i] + kd * md * prevP[i-1]
        dP[i] = newP

    return [dXd, dT] + dP

def check_coef(t, y, terminal=False):
    return 1 - y[0]

def block2(P1, e: BaseExperiment, time):
    # Время интегрирования
    timeStart = time
    timeEnd = 250
    check_coef.terminal = True
    check_coef.direction = 1

    y0 = [0, e.T1] + P1
    sol = sci.solve_ivp(
        kimpt2,
        [timeStart, timeEnd],
        y0,
        method="DOP853",
        args=(e,),
        events=check_coef)

    #sol = RKSolver(kimpt2, [timeStart, timeEnd], y0, args=(e,))

    MMn = []

    print(sol.y[0][-1], sol.y[1][-1], [ar[-1] for ar in sol.y[2:]], sep="\t| ")

    dt = (sol.t[-1]-sol.t[0])/len(sol.t)
    print(f'Отклонение температуры на стадии 2: {sum([(sol.y[1][int(i*2/dt)]-e.Texp2[i]) for i in range(len(e.Texp2))])}')
    return sol, MMn


In [361]:
# Среднечисленная молекулярная масса:
def getMn(e: BaseExperiment, P, MMn):
    numerator = 0
    denumerator = 0
    for i in range(e.FACTIONS):
        numerator += P[i] * MMn[i]
        denumerator += P[i]
    return numerator / denumerator

# Средневзвешенная молекулярная масса:
def getMv(e: BaseExperiment, P, MMn):
    numerator = 0
    denumerator = 0
    for i in range(e.FACTIONS):
        numerator += P[i] * MMn[i]**2
        denumerator += P[i] * MMn[i]
    return numerator / denumerator

# Среднеквадратическое отклонение ММР
def getStdVar(e: BaseExperiment, P, MMn):
    sum = 0
    Mn = getMn(e, P, MMn)
    for i in range(e.FACTIONS):
        sum = (e.MMs * P[i] - Mn * P[e.FACTIONS / 2])**2 # почему FACTIONS / 2 ?
    return math.sqrt(sum/(e.FACTIONS - 1))

def getA(e: BaseExperiment, P, MM, dP):
    a = []
    for n in range(len(P)):
        mn = int(e.Mr / dP)
        for _ in range(mn):
            a.append(e.Jacd1)
    return np.array(a)

def getB(A):
    N = len(A)
    b = []
    for _ in range(N):
        S = 0
        for _ in range(4):
            i = np.random.randint(N)
            S += S + A[i]
            np.delete(A, i)
        b.append(S)

        if len(A) < 4:
            break
    return np.array(b)

def getC(B, MM, N, dP, dMM):
    C = np.zeros((N, 2))
    for i in range(N):
        sum = 0
        NN = 1
        for j in range(len(B)):
            if (B[j] > MM[i]) and (B[j] <= MM[i] + dMM):
                sum = sum + B[j]
                NN += 1

        C[i][0] = sum / NN
        C[i][1] = NN * dP

    # общее количество повторений процедуры усреднения
    K = 4
    c = np.zeros((len(C), 2))
    for k in range(K):
        for i in range(len(C) - 1):
            c[i][0] = (C[i][0] + C[i + 1][0]) / 2
            c[i][1] = (C[i][1] + C[i + 1][1]) / 2
    return C

def calcMatrixC(e: BaseExperiment, P, MMn, dP=3, dMM=3):
    A = getA(e, P, MMn, dP)
    B = getB(A)
    C = getC(B, MMn, len(P), dP, dMM)
    return C

def printCharacteristics(e: BaseExperiment, C):
    pp = C[:, 0]
    mmn = C[:, 1]
    Mn = getMn(e, pp, mmn)
    Mv = getMv(e, pp, mmn)
    sigma = getStdVar(e, pp, mmn)

    print(f"Mn: {Mn}; Mv: {Mv}; sigma: {sigma}")
    return C

In [362]:
experiment = Experiment1()
sol1 = block1(experiment)

sol2, MMn = block2([ar[-1] for ar in sol1.y[3:]], experiment, sol1.t[-1])

P2 = [ar[-1] for ar in sol2.y[2:]]

#C = calcMatrixC(P2[len(P2)-1], MMn[len(MMn)-1])
#printCharacteristics(C)
#
#plt.subplot(1, 2, 1)
#plt.plot(C[550:750, 0], C[550:750, 1])
#plt.title("результат имитационного модели ММР")
#
#plt.subplot(1, 2, 2)
#plt.plot(MMn[5:35], P2[5:35], "r")  #
#plt.title("Рассчитанный модель")
#plt.show()

0.9999421841119783	| 0.9999998880357973	| 307.92223132489227	| [1.3340223087830528e-08, 1.3885572229485212e-08, 1.453333394352333e-08, 1.5237919039172406e-08, 1.5991253055290392e-08, 1.6791746870688143e-08, 1.7640018863115092e-08, 1.8537680234986095e-08, 1.948691143749926e-08, 2.049028134552395e-08, 2.1550670787253547e-08, 2.2671234253369323e-08, 2.3855385976147868e-08, 2.5106798892143576e-08, 2.6429406894147545e-08, 2.782741653594764e-08, 2.9305317903623693e-08, 3.0867900445733006e-08, 3.252027083382764e-08, 3.426787114809492e-08, 3.6116501662741674e-08, 3.807234284042709e-08, 4.014198112374828e-08, 4.233243614226328e-08, 4.4651189371377094e-08, 4.710621652432714e-08, 4.970602055716649e-08, 5.245966863913601e-08, 5.537683111601851e-08, 5.846782326315716e-08, 6.174365095423644e-08, 6.521605854652818e-08, 6.889758145867119e-08, 7.280160192190443e-08, 7.694240907942904e-08, 8.133526394567289e-08, 8.59964685161868e-08, 9.0943440905423e-08, 9.619479549421164e-08, 1.0177042946575274e-07, 1.

In [ ]:
def previewCharts(e: BaseExperiment, time1, t1, time2, t2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(time1[0], time1[-1], 1), y=e.Texp1, mode='markers', name='Экспериментальная температура',
                            marker=dict(color='blue', size=8)))
    fig.add_trace(go.Scatter(x=time1, y=t1, mode='lines', name='Температура модели',
                            line=dict(color='red', width=2)))
    fig.add_trace(go.Scatter(x=np.arange(time2[0], time2[-1], 2), y=e.Texp2, mode='markers', name='Экспериментальная температура',
                            marker=dict(color='blue', size=8)))
    fig.add_trace(go.Scatter(x=time2, y=t2, mode='lines', name='Температура модели',
                            line=dict(color='red', width=2)))
    fig.update_layout(
        title="Сравнение экспериментальной и температуры модели",
        xaxis_title="Time (s)",
        yaxis_title="Temperature (°C)",
        legend=dict(title="Data"),
        template="plotly_white"
    )
    fig.show()

def mmrChart(e: BaseExperiment, sol1, sol2):
    dN = e.ALL_FACTIONS/e.FACTIONS
    x = np.arange(dN, e.ALL_FACTIONS+1, dN)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=[ar[-1] for ar in sol1.y[3:]], name='Концентрация стадия 1', line_shape='hvh'))
    fig.add_trace(go.Scatter(x=x, y=[ar[-1] for ar in sol2.y[2:]], name='Концентрация стадия 2', line_shape='hvh'))
    fig.update_layout(
        title="ММР",
        yaxis=dict(exponentformat = 'power'),
        xaxis_title="г/моль",
        yaxis_title="моль/л",
        legend=dict(title="Data"),
        template="plotly_white"
    )
    fig.show()

print(len(sol1.t))

mmrChart(experiment, sol1, sol2)

previewCharts(experiment, sol1.t, sol1.y[2], sol2.t, sol2.y[1])